In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/My Drive/data")
import pandas as pd
import numpy as np
dat = pd.read_csv("sentiment_dtm_train.csv", encoding='CP949')

Catboost

In [4]:
! pip install catboost

     |████████████████████████████████| 66.2MB 43kB/s 


In [5]:
from catboost import CatBoostClassifier

In [10]:
cb_model = CatBoostClassifier(iterations=1000,
                          loss_function='Logloss',
                          eval_metric='Accuracy'
                          ,task_type="GPU"
                          )

In [15]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [ ]:
y = dat['label']
X = dat.drop(['label','Id'], axis=1)

In [23]:
accuracy = []
kf = KFold(n_splits=5, shuffle=True, random_state=0)
for i, (train_index,test_index) in enumerate(kf.split(dat)):
        y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
        X_train, X_valid = X.iloc[train_index, :], X.iloc[test_index,:]
        cb_model.fit(X_train, y_train)
        y_pred = cb_model.predict(X_valid)
        tmp_accuracy = accuracy_score(y_valid, y_pred)
        accuracy.append(tmp_accuracy)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5:	learn: 0.6495173	total: 302ms	remaining: 50s
6:	learn: 0.6501785	total: 342ms	remaining: 48.6s
7:	learn: 0.6579817	total: 379ms	remaining: 47s
8:	learn: 0.6668430	total: 417ms	remaining: 45.9s
9:	learn: 0.6665785	total: 449ms	remaining: 44.5s
10:	learn: 0.6667108	total: 481ms	remaining: 43.3s
11:	learn: 0.6768946	total: 521ms	remaining: 42.9s
12:	learn: 0.6764978	total: 556ms	remaining: 42.2s
13:	learn: 0.6768946	total: 587ms	remaining: 41.3s
14:	learn: 0.6767623	total: 625ms	remaining: 41s
15:	learn: 0.6772914	total: 656ms	remaining: 40.3s
16:	learn: 0.6803333	total: 687ms	remaining: 39.7s
17:	learn: 0.6798043	total: 718ms	remaining: 39.1s
18:	learn: 0.6800688	total: 751ms	remaining: 38.8s
19:	learn: 0.6799365	total: 786ms	remaining: 38.5s
20:	learn: 0.6809946	total: 818ms	remaining: 38.1s
21:	learn: 0.6843010	total: 849ms	remaining: 37.7s
22:	learn: 0.6845655	total: 894ms	remaining: 38s
23:	learn: 0.6869462	total: 939ms	remaining: 38.2s
24:	lear

In [25]:
np.mean(accuracy) # 0.7837495907934828

0.7837495907934828

LGBM

In [ ]:
params = {'max_depth': [10, 12, 14, 16],
          'learning_rate': [0.01, 0.05, 0.1],
          'min_child_samples': [50, 60, 40],
          'bagging_fraction' : [0.05, 0.1, 0.15],
            }


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import itertools


In [ ]:
def myfunc(**args):
    print(args)
keys = list(params)

parameter_df = pd.DataFrame(columns = keys)
parameter_df['Accuracy']=0
kf = KFold(n_splits=5, shuffle=True, random_state=0)
for i, vals in enumerate(itertools.product(*params.values())):
    param_comb = dict(zip(params, vals))
    parameter_df = pd.concat([parameter_df, pd.DataFrame.from_dict([param_comb])], ignore_index=True)
    accuracy = []
    for k, (train_index,test_index) in enumerate(kf.split(dat)):
      y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
      X_train, X_valid = X.iloc[train_index, :], X.iloc[test_index,:]
      lgbm = LGBMClassifier(**param_comb)
      lgbm.fit(X_train, y_train,  eval_metric='logloss',verbose=True)
      pred = lgbm.predict(X_valid)
      true = y_valid
      tmp_accuracy = accuracy_score(true, pred)
      accuracy.append(tmp_accuracy)
 
    parameter_df.loc[i, 'Accuracy'] = np.mean(accuracy)


In [ ]:
idx = [int(parameter_df['Accuracy'].idxmax()),]
best_param = parameter_df.iloc[idx, :-1].to_dict('list')
parameter_df.iloc[idx,]
# 0.781739
